In [4]:
%run __init__.py
from astropy.io import fits
from astropy.time import Time
from astropy.wcs import WCS
from astroquery.jplhorizons import Horizons
from scipy.ndimage import rotate
#from before_stack import *
from tools import *
from _mypath import thisdir
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [ ]:
# Borisov observed by Swift UVOT in Sep 2019
horizon_id  = 90004423
input_rela  = '../data/Borisov_raw/'
input_path  = get_path(input_rela)
map_type    = 'sk'
output_rela = '../docs/obs-log_Borisov_ZX.txt'
output_path = get_path(output_rela)

make_obs_log(horizon_id,
             input_path,
             map_type,
             output_path)

In [169]:
#### Here is an example of set_coord() and rotate()
# create an orignal array
image_array_test = np.array([[1 ,5, 6, 1],
                             [2, 9, 0, 6],
                             [7, 7, 5, 4]])
target_index_test = np.array([0, 0]) #get target index
new_coord_test = set_coord(image_array_test, target_index_test, (5, 5)) # shift the original array
rotated_image_test = rotate(new_coord_test, angle=40., reshape=False, order=1) # rotate the shifted array
rotated_image_test = np.around(rotated_image_test, decimals=2) # simplify the output
print("original image array:")
print(image_array_test)
print("\ntarget index:")
print(target_index_test)
print("\nshift the target to the center:")
print(new_coord_test)
print("\nrotate the image for 40 deg:")
print(rotated_image_test)

original image array:
[[1 5 6 1]
 [2 9 0 6]
 [7 7 5 4]]

target index:
[0 0]

shift the target to the center:
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 5. 6. 1. 0.]
 [0. 0. 0. 0. 2. 9. 0. 6. 0.]
 [0. 0. 0. 0. 7. 7. 5. 4. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

rotate the image for 40 deg:
[[0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.64 1.44 1.24 0.  ]
 [0.   0.   0.   0.   0.84 4.74 3.01 5.39 1.29]
 [0.   0.   0.   0.08 1.   6.18 4.7  4.49 1.61]
 [0.   0.   0.   0.   0.63 4.55 6.64 1.74 0.  ]
 [0.   0.   0.   0.   0.   2.78 1.28 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]]


In [171]:
def plot_stack(img_name, out_name):
    # read data from fits file
    img_path = get_path('../docs/'+img_name+'.fits')
    img_data = fits.open(img_path)[0].data
    # generate title by output name automatically
    out_path = get_path('../docs/'+out_name+'.png')
    title_name = out_name[:-4].split('_')
    title_name = ' '.join(title_name)
    # plot the stacked image
    fig = plt.figure(figsize=(30,30))
    ax = fig.add_subplot(111)
    plt.grid(alpha=0.3, ls='--')
    plt.imshow(img_data, vmin=0, vmax=np.max(img_data)*0.01)
    cbar = plt.colorbar(aspect=30, shrink=0.82)
    cbar.ax.tick_params(labelsize=20)
    ax.tick_params(axis='both', which='major', labelsize=20)
    plt.title(title_name, fontsize=30)
    plt.savefig(out_path, dpi=100)
#    plt.show()

In [172]:
def compare_with_DB(my_stack, DB_stack, save_name):
    my_path = get_path('../docs/'+my_stack)
    DB_path = get_path('../docs/'+DB_stack)
    save_path = get_path('../docs/'+save_name)
    my_data = fits.open(my_path)[0].data
    DB_data = fits.open(DB_path)[0].data.T[:1400, :1400]
    DB_px = fits.open(DB_path)[0].header['XPOS']
    DB_py = fits.open(DB_path)[0].header['YPOS']
    DB_data = set_coord(DB_data, np.array([DB_px, DB_py]), (1000, 1000))
    sub = my_data - DB_data
    hdu = fits.PrimaryHDU(sub)
    hdu.writeto(save_path)

In [ ]:
name = 'stacked_img'
filt = ('uw1', 'UVW1')
# filt = ('uvv', 'V')
# filt = ('uw2', 'UVW2')
stack_image('obs-log_Borisov.txt', filt[0], (1000,1000), name)
#plot_stack(img_name = name+'_'+filt[0], 
#           out_name = name+'_'+filt[0])
my_stack = name+'_'+filt[0]+'.fits'
DB_stack = 'sum_0_'+filt[1]+'.fits'
#compare_with_DB(my_stack, DB_stack, 'stack_compare_'+filt[1]+'.fits')